### Transform LSP 3D Keypoints To SMPL Model

#### 1. SMPL Joints Name:

```python
{
    0: 'Pelvis', 1: 'L_Hip', 2: 'R_Hip', 3: 'Spine1',
    4: 'L_Knee', 5: 'R_Knee', 6: 'Spine2', 7: 'L_Ankle',
    8: 'R_Ankle', 9: 'Spine3', 10: 'L_Foot', 11: 'R_Foot',
    12: 'Neck', 13: 'L_Collar', 14: 'R_Collar', 15: 'Head',
    16: 'L_Shoulder', 17: 'R_Shoulder', 18: 'L_Elbow', 19: 'R_Elbow',
    20: 'L_Wrist', 21: 'R_Wrist', 22: 'L_Hand', 23: 'R_Hand'
}
```

#### 2. LSP 3D Keypoints:

```python
{
    0: 'Head', 1: 'Neck', 2: 'R_Shoulder',
    3: 'R_Elbow', 4: 'R_Wrist', 5: 'L_Shoulder',
    6: 'L_Elbow', 7: 'L_Left', 8: 'R_Hip',
    9: 'R_Knee', 10: 'R_Ankle', 11: 'L_Hip',
    12: 'L_Knee', 13: 'L_Ankle'
}
```

In [1]:
import scipy.io
import tensorflow as tf
import numpy as np
import open3d as o3d
import pickle
import time

In [2]:
def rodrigues(r):
  """
  Rodrigues' rotation formula that turns axis-angle tensor into rotation
  matrix in a batch-ed manner.

  Parameter:
  ----------
  r: Axis-angle rotation tensor of shape [batch_size, 1, 3].

  Return:
  -------
  Rotation matrix of shape [batch_size, 3, 3].

  """
  theta = tf.norm(r + tf.random_normal(r.shape, 0, 1e-8, dtype=tf.float64), axis=(1, 2), keepdims=True)
  # avoid divide by zero
  r_hat = r / theta
  cos = tf.cos(theta)
  z_stick = tf.zeros(theta.get_shape().as_list()[0], dtype=tf.float64)
  m = tf.stack(
    (z_stick, -r_hat[:, 0, 2], r_hat[:, 0, 1], r_hat[:, 0, 2], z_stick,
     -r_hat[:, 0, 0], -r_hat[:, 0, 1], r_hat[:, 0, 0], z_stick), axis=1)
  m = tf.reshape(m, (-1, 3, 3))
  i_cube = tf.expand_dims(tf.eye(3, dtype=tf.float64), axis=0) + tf.zeros(
    (theta.get_shape().as_list()[0], 3, 3), dtype=tf.float64)
  A = tf.transpose(r_hat, (0, 2, 1))
  B = r_hat
  dot = tf.matmul(A, B)
  R = cos * i_cube + (1 - cos) * dot + tf.sin(theta) * m
  return R


def with_zeros(x):
  """
  Append a [0, 0, 0, 1] tensor to a [3, 4] tensor.

  Parameter:
  ---------
  x: Tensor to be appended.

  Return:
  ------
  Tensor after appending of shape [4,4]

  """
  ret = tf.concat(
    (x, tf.constant([[0.0, 0.0, 0.0, 1.0]], dtype=tf.float64)),
    axis=0
  )
  return ret


def pack(x):
  """
  Append zero tensors of shape [4, 3] to a batch of [4, 1] shape tensor.

  Parameter:
  ----------
  x: A tensor of shape [batch_size, 4, 1]

  Return:
  ------
  A tensor of shape [batch_size, 4, 4] after appending.

  """
  ret = tf.concat(
    (tf.zeros((x.get_shape().as_list()[0], 4, 3), dtype=tf.float64), x),
    axis=2
  )
  return ret


def smpl_model(model_path, betas, pose, trans, simplify=False):
  """
  Construct a compute graph that takes in parameters and outputs a tensor as
  model vertices. Face indices are also returned as a numpy ndarray.

  Parameters:
  ---------
  pose: Also known as 'theta', a [24,3] tensor indicating child joint rotation
  relative to parent joint. For root joint it's global orientation.
  Represented in a axis-angle format.

  betas: Parameter for model shape. A tensor of shape [10] as coefficients of
  PCA components. Only 10 components were released by SMPL author.

  trans: Global translation tensor of shape [3].

  Return:
  ------
  A tensor for vertices, and a numpy ndarray as face indices.

  """
  # For detailed comments see smpl_np.py
  with open(model_path, 'rb') as f:
    params = pickle.load(f)

  J_regressor = tf.constant(
    np.array(params['J_regressor'].todense(),
    dtype=np.float64)
  )
  weights = tf.constant(params['weights'], dtype=np.float64)
  posedirs = tf.constant(params['posedirs'], dtype=np.float64)
  v_template = tf.constant(params['v_template'], dtype=np.float64)
  shapedirs = tf.constant(params['shapedirs'], dtype=np.float64)
  f = params['f']

  kintree_table = params['kintree_table']
  id_to_col = {kintree_table[1, i]: i for i in range(kintree_table.shape[1])}
  parent = {
    i: id_to_col[kintree_table[0, i]]
    for i in range(1, kintree_table.shape[1])
  }
  v_shaped = tf.tensordot(shapedirs, betas, axes=[[2], [0]]) + v_template
  J = tf.matmul(J_regressor, v_shaped)
  pose_cube = tf.reshape(pose, (-1, 1, 3))
  R_cube_big = rodrigues(pose_cube)
  if simplify:
    v_posed = v_shaped
  else:
    R_cube = R_cube_big[1:]
    I_cube = tf.expand_dims(tf.eye(3, dtype=tf.float64), axis=0) + \
             tf.zeros((R_cube.get_shape()[0], 3, 3), dtype=tf.float64)
    lrotmin = tf.squeeze(tf.reshape((R_cube - I_cube), (-1, 1)))
    v_posed = v_shaped + tf.tensordot(posedirs, lrotmin, axes=[[2], [0]])
  results = []
  results.append(
    with_zeros(tf.concat((R_cube_big[0], tf.reshape(J[0, :], (3, 1))), axis=1))
  )
  for i in range(1, kintree_table.shape[1]):
    results.append(
      tf.matmul(
        results[parent[i]],
        with_zeros(
          tf.concat(
            (R_cube_big[i], tf.reshape(J[i, :] - J[parent[i], :], (3, 1))),
            axis=1
          )
        )
      )
    )
  stacked = tf.stack(results, axis=0)
  results = stacked - \
            pack(
              tf.matmul(
                stacked,
                tf.reshape(
                  tf.concat((J, tf.zeros((24, 1), dtype=tf.float64)), axis=1),
                  (24, 4, 1)
                )
              )
            )
  T = tf.tensordot(weights, results, axes=((1), (0)))
  rest_shape_h = tf.concat(
    (v_posed, tf.ones((v_posed.get_shape().as_list()[0], 1), dtype=tf.float64)),
    axis=1
  )
  v = tf.matmul(T, tf.reshape(rest_shape_h, (-1, 4, 1)))
  v = tf.reshape(v, (-1, 4))[:, :3]
  result = v + tf.reshape(trans, (1, 3))
  return result, f

In [3]:
def isRotationMatrix(R):
    # square matrix test
    if R.ndim != 2 or R.shape[0] != R.shape[1]:
        return False
    should_be_identity = np.allclose(R.dot(R.T), np.identity(R.shape[0], np.float))
    should_be_one = np.allclose(np.linalg.det(R), 1)
    return should_be_identity and should_be_one

def calAxisXYZ(a, b, need_R=False):
    rotationAxis = np.cross(a, b)
    rotationAngle = np.arccos(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))
    
    norm = np.linalg.norm(rotationAxis)
    rotationMatrix = np.zeros([3, 3])
    if norm == 0:
        wx, wy, wz =0., 0., 0.
    else:
        norm_rotationAxis = [i / norm for i in rotationAxis]
        wx, wy, wz = norm_rotationAxis
    sin, cos = np.sin(rotationAngle), np.cos(rotationAngle)
    
    rotationMatrix[0][0] = cos + (wx ** 2) * (1 - cos)
    rotationMatrix[0][1] = wx*wy*(1 - cos) - wz*sin
    rotationMatrix[0][2] = wy*sin + wx*wz*(1 - cos)
    
    rotationMatrix[1][0] = wz*sin + wx*wy*(1 - cos)
    rotationMatrix[1][1] = cos + (wy ** 2) * (1 - cos)
    rotationMatrix[1][2] = wy*wz*(1 - cos) - wx*sin
    
    rotationMatrix[2][0] = wx*wz*(1 - cos) - wy*sin
    rotationMatrix[2][1] = wx*sin + wy*wz*(1 - cos)
    rotationMatrix[2][2] = cos + (wz ** 2) * (1 - cos)
    
    ax = np.arctan2(rotationMatrix[2][1], rotationMatrix[2][2])
    ay = np.arctan2(-rotationMatrix[2][0], np.sqrt(rotationMatrix[2][1] ** 2 + rotationMatrix[2][2] ** 2))
    az = np.arctan2(rotationMatrix[1][0], rotationMatrix[0][0])
    
    if not need_R:
        return ax, ay, az
    else:
        return ax, ay, az, rotationMatrix

In [4]:
def rotationPose3d(pose_3d, R):
    r_pose_3d = np.zeros(pose_3d.shape)
    
    # If R is rotation A->B, then A*R = B, * is dot.
    for i in range(14):
        r_pose_3d[i] = np.dot(R, pose_3d[i])
        
    return r_pose_3d

def pose2smpl(pose_3d):
    
    smpl_pose = np.zeros([24, 3])
    
    def ab2smpl(a, b):
        ax, ay, az = calAxisXYZ(a, b)
        smpl = -ax, -ay, az
        return smpl
    
    # Right hip
    a = [0, 1, 0]
    b = pose_3d[9] - pose_3d[8]
    smpl_pose[2] = ab2smpl(a, b)
    
    # Right knee
    a = pose_3d[9] - pose_3d[8]
    b = pose_3d[10] - pose_3d[9]
    smpl_pose[5] = ab2smpl(a, b)
    
    # Left hip
    a = [0, 1, 0]
    b = pose_3d[12] - pose_3d[11]
    smpl_pose[1] = ab2smpl(a, b)
    
    # Left knee
    a = pose_3d[12] - pose_3d[11]
    b = pose_3d[13] - pose_3d[12]
    smpl_pose[4] = ab2smpl(a, b)
    
    # Right shoulder
    a = pose_3d[8] - pose_3d[11]
    b = pose_3d[2] - pose_3d[1]
    smpl_pose[14] = ab2smpl(a, b)
    
    # Right elbow
    a = pose_3d[2] - pose_3d[1]
    b = pose_3d[3] - pose_3d[2]
    smpl_pose[17] = ab2smpl(a, b)
    
    # Right hand
    a = pose_3d[3] - pose_3d[2]
    b = pose_3d[4] - pose_3d[3]
    smpl_pose[19] = ab2smpl(a, b)
    
    # Left shoulder
    a = pose_3d[11] - pose_3d[8]
    b = pose_3d[5] - pose_3d[1]
    smpl_pose[13] = ab2smpl(a, b)

    # Left elbow
    a = pose_3d[5] - pose_3d[1]
    b = pose_3d[6] - pose_3d[5]
    smpl_pose[16] = ab2smpl(a, b)
    
    # Left hand
    a = pose_3d[6] - pose_3d[5]
    b = pose_3d[7] - pose_3d[6]
    smpl_pose[18] = ab2smpl(a, b)
    
    # smpl[3]
    a = [0, -1, 0]
    b = pose_3d[1]
    smpl_pose[3] = ab2smpl(a, b)
    
    # neck
    a = [0, -1, 0]
    b = pose_3d[1]
    smpl_pose[12] = ab2smpl(a, b)
    
    # head
    a = pose_3d[1]
    b = pose_3d[0] - pose_3d[1]
    smpl_pose[15] = ab2smpl(a, b)
    
    smpl_pose = smpl_pose.reshape([72])
    
    return smpl_pose

In [5]:
lsp_data_file = '/home/commaai-03/Mikoy/work/workspace/SMPL-master/3D_library.mat'
lsp_data = scipy.io.loadmat(lsp_data_file)

In [6]:
all_pose_3d = lsp_data['s1_s9_3d'].reshape(-1, 14 ,3)

In [ ]:
betas = np.zeros(10)
trans = np.zeros(3)
model_file = '/home/commaai-03/Mikoy/work/workspace/SMPL-master/model.pkl'
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

'''
# For open3D
vis = o3d.visualization.Visualizer()
vis.create_window('SMPL', 1280, 720)
mesh = o3d.geometry.TriangleMesh()
vis.add_geometry(mesh)
'''
index = 0

try:
    for pose_3d in all_pose_3d:
        t0 = time.time()
        center = (pose_3d[8] + pose_3d[11]) / 2.
        trans_pose_3d = (pose_3d - center) / 1000.

        a = trans_pose_3d[8]
        b = [1, 0, 0]
        _, _, _, R = calAxisXYZ(a, b, need_R=True)
        r_pose_3d = rotationPose3d(trans_pose_3d, R)

        smpl_pose = pose2smpl(r_pose_3d)
        t1 = time.time()

        # For tensorflow
        _pose = tf.constant(smpl_pose, dtype=tf.float64)
        _betas = tf.constant(betas, dtype=tf.float64)
        _trans = tf.constant(trans, dtype=tf.float64)
        output, faces = smpl_model(model_file, _betas, _pose, _trans, True)
        result = sess.run(output)
        t2 = time.time()
        
        mesh = o3d.geometry.TriangleMesh()
        mesh.vertices = o3d.utility.Vector3dVector(result)
        mesh.triangles = o3d.utility.Vector3iVector(faces)
        mesh.compute_vertex_normals()
        mesh.paint_uniform_color([0.3, 0.3, 0.3])
        t3 = time.time()
        
        o3d.visualization.draw_geometries([mesh], 'SMPL: %d' % index, 1280, 720)
        index += 1
        
        '''
        # Update windows
        vis.update_geometry(mesh)
        vis.poll_events()
        vis.update_renderer()
        '''
        t4 = time.time()

        print('[T1]: %.5f | [T2]: %.5f | [T3]: %.5f | [T4]: %.5f'
               % (t1-t0, t2-t1, t3-t2, t4-t3))
finally:
    sess.close()
    #vis.destroy_window()

[T1]: 0.00091 | [T2]: 0.88844 | [T3]: 0.00602 | [T4]: 11.44342
[T1]: 0.00135 | [T2]: 0.38363 | [T3]: 0.00619 | [T4]: 3.99577
[T1]: 0.00145 | [T2]: 0.39064 | [T3]: 0.00634 | [T4]: 6.74138
[T1]: 0.00167 | [T2]: 0.40362 | [T3]: 0.00610 | [T4]: 3.95989
[T1]: 0.00129 | [T2]: 0.42167 | [T3]: 0.00657 | [T4]: 1.36903
[T1]: 0.00126 | [T2]: 0.47536 | [T3]: 0.00648 | [T4]: 1.94199
[T1]: 0.00133 | [T2]: 0.44673 | [T3]: 0.00667 | [T4]: 1.59570
[T1]: 0.00157 | [T2]: 0.46077 | [T3]: 0.00645 | [T4]: 1.52162
[T1]: 0.00158 | [T2]: 0.47469 | [T3]: 0.00623 | [T4]: 3.00208
[T1]: 0.00140 | [T2]: 0.48962 | [T3]: 0.00669 | [T4]: 2.38932
[T1]: 0.00152 | [T2]: 0.50523 | [T3]: 0.00653 | [T4]: 1.47681
[T1]: 0.00148 | [T2]: 0.52426 | [T3]: 0.00622 | [T4]: 2.06573
[T1]: 0.00200 | [T2]: 0.53764 | [T3]: 0.00647 | [T4]: 2.94185
[T1]: 0.00154 | [T2]: 0.54598 | [T3]: 0.00609 | [T4]: 2.03302
[T1]: 0.00132 | [T2]: 0.57086 | [T3]: 0.00661 | [T4]: 1.32147
[T1]: 0.00148 | [T2]: 0.62744 | [T3]: 0.00626 | [T4]: 1.04039
[T1]: 0

In [ ]:
betas = np.zeros(10)
trans = np.zeros(3)
model_file = '/home/commaai-03/Mikoy/work/workspace/SMPL-master/model.pkl'
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [ ]:
pose_3d = all_pose_3d[6001]
center = (pose_3d[8] + pose_3d[11]) / 2.
trans_pose_3d = (pose_3d - center) / 1000.

a = trans_pose_3d[8]
b = [1, 0, 0]
_, _, _, R = calAxisXYZ(a, b, need_R=True)
r_pose_3d = rotationPose3d(trans_pose_3d, R)

smpl_pose = pose2smpl(r_pose_3d)

# For tensorflow
_pose = tf.constant(smpl_pose, dtype=tf.float64)
_betas = tf.constant(betas, dtype=tf.float64)
_trans = tf.constant(trans, dtype=tf.float64)
output, faces = smpl_model(model_file, _betas, _pose, _trans, True)
result = sess.run(output)

mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(result)
mesh.triangles = o3d.utility.Vector3iVector(faces)
mesh.compute_vertex_normals()
mesh.paint_uniform_color([0.3, 0.3, 0.3])

o3d.visualization.draw_geometries([mesh])